In [23]:
import geopandas as gpd
import pandas as pd
import warnings
# Apagar todos los warnings
warnings.filterwarnings("ignore")

load_block_cols = ['MANZENT_I', 'Number_res', 'Number_hou', 'ID_PLACA', 'geometry']

In [55]:
# Actuales
gdf = gpd.read_file('/app/data/manzanas_hab/Manzanas base')
# [load_block_cols]
gdf['pers'] = gdf[['Number_res', 'TOTAL_PERS']].max(axis=1)
traslate = {
    'MANZENT_I': 'block_id',
    'pers': 'q_residents',
}
gdf.rename(columns=traslate, inplace=True)
gdf['ID_PLACA'].fillna(0,inplace=True)
save_cols = ['block_id', 'q_residents', 'geometry']
gdf = gdf[save_cols]
gdf = gpd.GeoDataFrame(data=gdf.drop(columns=['geometry']), geometry=gdf['geometry'])
gdf.to_parquet('/app/assets/blocks/actual.parquet')

In [60]:
gdf = gpd.read_file('/app/data/manzanas_actuales_futuro/future')
# [load_block_cols]
traslate = {
    'MANZENT_I': 'block_id',
    'Number_res': 'q_residents',
}
gdf.rename(columns=traslate, inplace=True)
gdf['ID_PLACA'].fillna(0,inplace=True)

def fix_ids(gdf):
    mask = gdf['block_id'].isna()
    not_id = gdf.loc[mask, :]
    yes_id = gdf.loc[~mask, :]
    not_id.loc[:, 'block_id'] = ['2024']*len(not_id) + not_id.index.astype(str)
    gdf = pd.concat([yes_id, not_id])
    return gdf

gdf = fix_ids(gdf)
gdf = gdf[save_cols]
gdf.to_parquet('/app/assets/blocks/future.parquet')